This notebook takes a look into the text data in `train_context.csv`, outlines a few possible improvements, and develops a cleaning pipeline.

In [305]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
import langdetect
from tqdm.notebook import tqdm
from iso639 import languages
from copy import deepcopy

In [23]:
topics = pd.read_csv("../../input/topics.csv")

In [233]:
train_context = pd.read_csv("../../input/train_context.csv")

In [3]:
train_context

,topics_ids,content_ids,topic_title,topic_description,content_title,content_description,content_text,topic_parent_title,topic_parent_description,topic_child_title,topic_child_description,target
0,t_3d9ad9931021,c_efb73ad83f4b,NaN,BC: BIOL 2 - Introduction to Human Biology (Gr...,NaN,NaN,Orientaciones para el profesorado\nOrientacion...,Butte College [SEP] Campus Courses [SEP] Libre...,NaN,Text [SEP] 2: Introduction to Human Biology [S...,NaN,0
1,t_3d9ad9931021,c_77574ef20c1f,NaN,BC: BIOL 2 - Introduction to Human Biology (Gr...,NaN,Recurso Educativo Abierto para Ciencias Natura...,"¿Es magia? No, son reacciones químicas\n¿Es ma...",Butte College [SEP] Campus Courses [SEP] Libre...,NaN,Text [SEP] 2: Introduction to Human Biology [S...,NaN,0
2,t_3d9ad9931021,c_200ae87baf4d,NaN,BC: BIOL 2 - Introduction to Human Biology (Gr...,NaN,Recurso Educativo Abierto para Geografía e His...,La economía y yo\nLa economía y yo\nObra publi...,Butte College [SEP] Campus Courses [SEP] Libre...,NaN,Text [SEP] 2: Introduction to Human Biology [S...,NaN,0
3,t_3d9ad9931021,c_87e171afe50b,NaN,BC: BIOL 2 - Introduction to Human Biology (Gr...,NaN,NaN,La curiosidad es saludable\nLa curiosidad es s...,Butte College [SEP] Campus Courses [SEP] Libre...,NaN,Text [SEP] 2: Introduction to Human Biology [S...,NaN,0
4,t_3d9ad9931021,c_3c070b63a944,NaN,BC: BIOL 2 - Introduction to Human Biology (Gr...,NaN,NaN,"Palabra de caballero\nPalabra de caballero\n""P...",Butte College [SEP] Campus Courses [SEP] Libre...,NaN,Text [SEP] 2: Introduction to Human Biology [S...,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
615165,t_70da08637930,c_eb6448437b5f,8.1.5 Use dot (.) and cross (x) diagrams to il...,Use dot (.) and cross (x) diagrams to illustra...,Level 2: Describe the formation of ionic bond ...,NaN,NaN,8.1 The role of outer electrons in chemical bo...,The role of outer electrons in chemical bondin...,8.1.5 Use dot (.) and cross (x) diagrams to il...,Materials in this folder have been reviewed by...,0
615166,t_70da08637930,c_07c1da15995b,8.1.5 Use dot (.) and cross (x) diagrams to il...,Use dot (.) and cross (x) diagrams to illustra...,Level 3: Describe the formation of ionic bond ...,NaN,NaN,8.1 The role of outer electrons in chemical bo...,The role of outer electrons in chemical bondin...,8.1.5 Use dot (.) and cross (x) diagrams to il...,Materials in this folder have been reviewed by...,0
615167,t_70da08637930,c_17ff16d31106,8.1.5 Use dot (.) and cross (x) diagrams to il...,Use dot (.) and cross (x) diagrams to illustra...,Level 1: Describe the formation of ionic bond ...,NaN,NaN,8.1 The role of outer electrons in chemical bo...,The role of outer electrons in chemical bondin...,8.1.5 Use dot (.) and cross (x) diagrams to il...,Materials in this folder have been reviewed by...,0
615168,t_70da08637930,c_7cb9a57f2219,8.1.5 Use dot (.) and cross (x) diagrams to il...,Use dot (.) and cross (x) diagrams to illustra...,"Ligações iônicas, covalentes e metálicas",Existem três tipos principais de ligações quím...,NaN,8.1 The role of outer electrons in chemical bo...,The role of outer electrons in chemical bondin...,8.1.5 Use dot (.) and cross (x) diagrams to il...,Materials in this folder have been reviewed by...,0


# Topic Titles

In [9]:
topic_titles = train_context.topic_title[~train_context.topic_title.isna()]

In [76]:
topic_titles.shape

(615160,)

In [33]:
topics.language.unique()

array(['bg', 'en', 'pt', 'gu', 'my', 'zh', 'ar', 'te', 'es', 'fr', 'sw',
       'mr', 'hi', 'bn', 'fil', 'ru', 'it', 'or', 'pnb', 'km', 'as', 'kn',
       'ur', 'pl', 'ta', 'swa', 'tr', 'mul'], dtype=object)

* `:`/`?`/`!`/`.`/`'`/`-`/`(`/`)`/`/`/`,`/`<`/`>`/`=`/`^`/`\n`
* standalone non-year numbers
* '1870-1939'
* `Du +Du =D`/`DU+U/U+DU/DU+U`
* 'Lección 31', 'Lección 34', 'Lección 17', 'Lección 32',
       'Convection', 'Lección 12', 'Lección 20', 'Lección 33',
       'Lección 16'
* case vs uncased
* remove useless words (small words)
* remove white space
* handle duplicates
    * remove duplicates within same tree
    * remove 
* for context, filter out if diff language

* Need to develop some way to check whether a title is important or not (same goes for all text columns).
* maybe account for length of text

In [90]:
topics.shape

(76972, 9)

In [86]:
topics.title.nunique()

45082

In [89]:
topics.title[topics.title.duplicated(keep=False)]

2                          Entradas e saídas de uma função
3                                              Transcripts
5                                        100 સુધીનો સરવાળો
7                                                    Nepal
12                          Addition and Subtraction to 20
                               ...                        
76964                                  Polynomial division
76967                               4.3 Graph of functions
76968                     Inscribed shapes problem solving
76969                                            Lección 7
76971    Formation of shadows and eclipses (umbra and p...
Name: title, Length: 44702, dtype: object

In [93]:
topics[topics.title=="Transcripts"]

,id,title,description,channel,category,level,language,parent,has_content
3,t_00069b63a70a,Transcripts,NaN,6e3ba4,source,3,en,t_4054df11a74e,True
466,t_01a382c3c11a,Transcripts,NaN,6e3ba4,source,4,en,t_a26c895cd483,True
493,t_01c3821eaa65,Transcripts,NaN,6e3ba4,source,4,en,t_a004e7d2f5f8,True
524,t_01dfb32014ce,Transcripts,NaN,6e3ba4,source,3,en,t_3c29c43c2b33,True
537,t_01e7baeee187,Transcripts,NaN,6e3ba4,source,4,en,t_548f6d1a9a58,True
...,...,...,...,...,...,...,...,...,...
75045,t_f9ac46f796db,Transcripts,NaN,6e3ba4,source,4,en,t_9955899f6555,True
75776,t_fc33cc6e3361,Transcripts,NaN,6e3ba4,source,4,en,t_cb298c56e5ed,True
76081,t_fd356b8f7f67,Transcripts,NaN,6e3ba4,source,3,en,t_28ca2feca84f,True
76697,t_ff1e2de126aa,Transcripts,NaN,6e3ba4,source,4,en,t_4c250eb922c0,True


There are 615160 rows in this subset of `train_context`. It only has 1 less topic id. There are only 36597 unique titles for topics. Why? I'd expect 1 unique title (or at worst, similar) for every topic id.

What do we do with these duplicated titles?

In [71]:
topic_titles.shape

(615160,)

In [69]:
start = int(input("Enter a start: "))

topic_titles.unique()[start:start+500]

Enter a start:  36500


array(['6.4.1 Formation of simple ions (cations and anions): qualitative treatment of the ionisation energy and electron affinity',
       '4: Frequency Response of First Order Systems, Transfer Functions, and General Method for Derivation of Frequency Response',
       'Myths and Scientific Explanations Behind Natural Phenomena - Mga Alamat at Siyentipikong Paliwanang ng mga Likas na Penome',
       'يتعرّف ويستنتج قانون (قاعدة) الجيب لأي مثلّث والتي تنص على: تتناسب أطوال الأضلاع  في أي مثلّث مع جيوب الزوايا المقابلة لها',
       'Unidad III: Investigando para profundizar Unidad de comprensión: El desarrollo del agua Currículum de conocimientos básicos',
       'Unidad II: Hacer una Unidad de Reclamos Basada en la Evidencia: Plato Desarrollar el Currículum de las Competencias Básicas',
       'B3.1.10.1.2. Use appropriate verbal and non-verbal cues to convey meaning while speaking, e.g. smiling, using gestures, etc',
       'B5.4.9.1: Apply the skills and strategies for idea generatio

In [84]:
train_context

,topics_ids,content_ids,topic_title,topic_description,content_title,content_description,content_text,topic_parent_title,topic_parent_description,topic_child_title,topic_child_description,target
0,t_3d9ad9931021,c_efb73ad83f4b,NaN,BC: BIOL 2 - Introduction to Human Biology (Gr...,NaN,NaN,Orientaciones para el profesorado\nOrientacion...,Butte College [SEP] Campus Courses [SEP] Libre...,NaN,Text [SEP] 2: Introduction to Human Biology [S...,NaN,0
1,t_3d9ad9931021,c_77574ef20c1f,NaN,BC: BIOL 2 - Introduction to Human Biology (Gr...,NaN,Recurso Educativo Abierto para Ciencias Natura...,"¿Es magia? No, son reacciones químicas\n¿Es ma...",Butte College [SEP] Campus Courses [SEP] Libre...,NaN,Text [SEP] 2: Introduction to Human Biology [S...,NaN,0
2,t_3d9ad9931021,c_200ae87baf4d,NaN,BC: BIOL 2 - Introduction to Human Biology (Gr...,NaN,Recurso Educativo Abierto para Geografía e His...,La economía y yo\nLa economía y yo\nObra publi...,Butte College [SEP] Campus Courses [SEP] Libre...,NaN,Text [SEP] 2: Introduction to Human Biology [S...,NaN,0
3,t_3d9ad9931021,c_87e171afe50b,NaN,BC: BIOL 2 - Introduction to Human Biology (Gr...,NaN,NaN,La curiosidad es saludable\nLa curiosidad es s...,Butte College [SEP] Campus Courses [SEP] Libre...,NaN,Text [SEP] 2: Introduction to Human Biology [S...,NaN,0
4,t_3d9ad9931021,c_3c070b63a944,NaN,BC: BIOL 2 - Introduction to Human Biology (Gr...,NaN,NaN,"Palabra de caballero\nPalabra de caballero\n""P...",Butte College [SEP] Campus Courses [SEP] Libre...,NaN,Text [SEP] 2: Introduction to Human Biology [S...,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
615165,t_70da08637930,c_eb6448437b5f,8.1.5 Use dot (.) and cross (x) diagrams to il...,Use dot (.) and cross (x) diagrams to illustra...,Level 2: Describe the formation of ionic bond ...,NaN,NaN,8.1 The role of outer electrons in chemical bo...,The role of outer electrons in chemical bondin...,8.1.5 Use dot (.) and cross (x) diagrams to il...,Materials in this folder have been reviewed by...,0
615166,t_70da08637930,c_07c1da15995b,8.1.5 Use dot (.) and cross (x) diagrams to il...,Use dot (.) and cross (x) diagrams to illustra...,Level 3: Describe the formation of ionic bond ...,NaN,NaN,8.1 The role of outer electrons in chemical bo...,The role of outer electrons in chemical bondin...,8.1.5 Use dot (.) and cross (x) diagrams to il...,Materials in this folder have been reviewed by...,0
615167,t_70da08637930,c_17ff16d31106,8.1.5 Use dot (.) and cross (x) diagrams to il...,Use dot (.) and cross (x) diagrams to illustra...,Level 1: Describe the formation of ionic bond ...,NaN,NaN,8.1 The role of outer electrons in chemical bo...,The role of outer electrons in chemical bondin...,8.1.5 Use dot (.) and cross (x) diagrams to il...,Materials in this folder have been reviewed by...,0
615168,t_70da08637930,c_7cb9a57f2219,8.1.5 Use dot (.) and cross (x) diagrams to il...,Use dot (.) and cross (x) diagrams to illustra...,"Ligações iônicas, covalentes e metálicas",Existem três tipos principais de ligações quím...,NaN,8.1 The role of outer electrons in chemical bo...,The role of outer electrons in chemical bondin...,8.1.5 Use dot (.) and cross (x) diagrams to il...,Materials in this folder have been reviewed by...,0


# Topic Description

In [117]:
topics

,id,title,description,channel,category,level,language,parent,has_content
0,t_00004da3a1b2,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True
1,t_000095e03056,Unit 3.3 Enlargements and Similarities,NaN,b3f329,aligned,2,en,t_aa32fb6252dc,False
2,t_00068291e9a4,Entradas e saídas de uma função,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True
3,t_00069b63a70a,Transcripts,NaN,6e3ba4,source,3,en,t_4054df11a74e,True
4,t_0006d41a73a8,Графики на експоненциални функции (Алгебра 2 н...,Научи повече за графиките на сложните показате...,000cf7,source,4,bg,t_e2452e21d252,True
...,...,...,...,...,...,...,...,...,...
76967,t_fffb0bf2801d,4.3 Graph of functions,NaN,e77b55,aligned,4,en,t_676e6a1a4dc7,False
76968,t_fffbe1d5d43c,Inscribed shapes problem solving,Use properties of inscribed angles to prove pr...,0c929f,source,4,sw,t_50145b9bab3f,True
76969,t_fffe14f1be1e,Lección 7,NaN,6e90a7,aligned,6,es,t_d448c707984d,True
76970,t_fffe811a6da9,تحديد العلاقة بين الإحداثيّات القطبية والإحداث...,5b9e5ca86571f90499ea987f,9fd860,source,2,ar,t_5b4f3ba4eb7d,True


In [70]:
topic_desc = train_context.topic_description[~train_context.topic_description.isna()]

In [121]:
topics[~topics.description.isna()].description.nunique()

23067

In [116]:
topic_desc.nunique()

18397

* what is this source id thing in the topic desc?
* how do we parse book titles?
* manage length
* special symbols again
* '5ad59f7a6b9064043e263f03', '5b9e5bd46571f90499ea8d69',
       '5a4c844e7dd197090857fce9'train_context.topic_description.apply(lambda x: True if "source" in x else False)
*  'નમસ્કાર, \nધો:- ૬ થી ૮ ગુજરાતીની ક્વિઝ અહિયા હુ રજૂ કરુ છું. આશા રાખુ કે આ ક્વિઝ તમને ઉપયોગી થશે. બીજી અન્ય ઉપયોગી માહિતિ મારા બ્લોગ અને યુ-ટ્યુબ ચેનલ પર જોવા  નીચેનીની લીંક પર જાઓ.                                          \n My Youtube Channel\nhttps://www.youtube.com/channel/UCQue-6mFDpz31MYWvrBDTUQ?sub_confirmation=1\nMy Blog\nhttp://mehuloza.blogspot.com/'

In [129]:
train_context.topic_description

0         BC: BIOL 2 - Introduction to Human Biology (Gr...
1         BC: BIOL 2 - Introduction to Human Biology (Gr...
2         BC: BIOL 2 - Introduction to Human Biology (Gr...
3         BC: BIOL 2 - Introduction to Human Biology (Gr...
4         BC: BIOL 2 - Introduction to Human Biology (Gr...
                                ...                        
615165    Use dot (.) and cross (x) diagrams to illustra...
615166    Use dot (.) and cross (x) diagrams to illustra...
615167    Use dot (.) and cross (x) diagrams to illustra...
615168    Use dot (.) and cross (x) diagrams to illustra...
615169    Use dot (.) and cross (x) diagrams to illustra...
Name: topic_description, Length: 615170, dtype: object

In [138]:
train_context.topic_description.apply(lambda x: x)

0         BC: BIOL 2 - Introduction to Human Biology (Gr...
1         BC: BIOL 2 - Introduction to Human Biology (Gr...
2         BC: BIOL 2 - Introduction to Human Biology (Gr...
3         BC: BIOL 2 - Introduction to Human Biology (Gr...
4         BC: BIOL 2 - Introduction to Human Biology (Gr...
                                ...                        
615165    Use dot (.) and cross (x) diagrams to illustra...
615166    Use dot (.) and cross (x) diagrams to illustra...
615167    Use dot (.) and cross (x) diagrams to illustra...
615168    Use dot (.) and cross (x) diagrams to illustra...
615169    Use dot (.) and cross (x) diagrams to illustra...
Name: topic_description, Length: 615170, dtype: object

In [148]:
tmp = train_context[train_context['topic_description'].str.contains('source_id').fillna(False)][["topics_ids", "topic_description"]]
tmp

,topics_ids,topic_description
680,t_9bf1d7db3cac,source_id=LES867
681,t_9bf1d7db3cac,source_id=LES867
682,t_9bf1d7db3cac,source_id=LES867
683,t_9bf1d7db3cac,source_id=LES867
684,t_9bf1d7db3cac,source_id=LES867
...,...,...
523535,t_b79b44d912cf,source_id=LES1546
523536,t_b79b44d912cf,source_id=LES1546
523537,t_b79b44d912cf,source_id=LES1546
523538,t_b79b44d912cf,source_id=LES1546


In [151]:
topics[["id", "category"]]

,id,category
0,t_00004da3a1b2,source
1,t_000095e03056,aligned
2,t_00068291e9a4,source
3,t_00069b63a70a,source
4,t_0006d41a73a8,source
...,...,...
76967,t_fffb0bf2801d,aligned
76968,t_fffbe1d5d43c,source
76969,t_fffe14f1be1e,aligned
76970,t_fffe811a6da9,source


In [153]:
tmp = tmp.merge(topics[["id", "category"]], how="right", left_on="topics_ids", right_on="id")
tmp[~tmp.topics_ids.isna()]

,topics_ids,topic_description,id,category
18,t_001558da4876,source_id=LES1116,t_001558da4876,source
19,t_001558da4876,source_id=LES1116,t_001558da4876,source
20,t_001558da4876,source_id=LES1116,t_001558da4876,source
21,t_001558da4876,source_id=LES1116,t_001558da4876,source
22,t_001558da4876,source_id=LES1116,t_001558da4876,source
...,...,...,...,...
84558,t_fff1047917af,source_id=LES1229,t_fff1047917af,source
84559,t_fff1047917af,source_id=LES1229,t_fff1047917af,source
84560,t_fff1047917af,source_id=LES1229,t_fff1047917af,source
84561,t_fff1047917af,source_id=LES1229,t_fff1047917af,source


In [157]:
tmp[~tmp.topics_ids.isna()][tmp.category=="aligned"]

/opt/conda/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,topics_ids,topic_description,id,category
9896,t_1edffdb53b30,source_id=LES1289,t_1edffdb53b30,aligned
9897,t_1edffdb53b30,source_id=LES1289,t_1edffdb53b30,aligned
9898,t_1edffdb53b30,source_id=LES1289,t_1edffdb53b30,aligned
9899,t_1edffdb53b30,source_id=LES1289,t_1edffdb53b30,aligned
9900,t_1edffdb53b30,source_id=LES1289,t_1edffdb53b30,aligned
9901,t_1edffdb53b30,source_id=LES1289,t_1edffdb53b30,aligned
9902,t_1edffdb53b30,source_id=LES1289,t_1edffdb53b30,aligned
9903,t_1edffdb53b30,source_id=LES1289,t_1edffdb53b30,aligned
9904,t_1edffdb53b30,source_id=LES1289,t_1edffdb53b30,aligned
9905,t_1edffdb53b30,source_id=LES1289,t_1edffdb53b30,aligned


Topics are of categories "source" or "aligned" for where topic descriptions are `source_id=LES####`.

In [159]:
start = int(input("Enter a start: "))

topic_desc.unique()[start:start+500]

Enter a start:  9000


array(['Maisha ni mazuri, ila yanaweza kuendelea kuwa bora. Kuwa na uwezo wa kutafuta namba ndogo ambayo ni kigawe cha namba nyingine mbili! Kinaitwa KDS au kigawe kidogo cha shirika. Zaidi ya kufanya maisha yako yajitosheleze KDS inaweza kukusaidia kufanya mambo zaidi kama kujumlisha sehemu. Viwango vy msingi kawaida: 6.NS.B.4',
       'Aprenda como definimos logaritmos e use essa definição para calcular diversos logaritmos. Por exemplo, calcule log_2(8) como 3, percebendo que 2^3=8.\n ',
       "Integers can be positive or negative whole numbers. Did you notice any use of negative integers in our day-to-day life? You are right! Canadian winter temperatures, of course! Let's get a visual understanding of positive and negative integers",
       'Como uma introdução à multiplicação polinomial mais complexa, aprenda a multiplicar monômios (que são polinômios com um só termo). Por exemplo, multiplique 2x³ e 5x⁷.',
       'Въпреки че може да е забавно да освободим от законите на физиката (

# Content Title

In [167]:
content_title = train_context.content_title[~train_context.content_title.isna()]

In [168]:
content_title

9                                                      લોલક
10                                            Uso de R y RR
11                                         As letras R e RR
12                                                 Letter R
13                                            Meet Letter R
                                ...                        
615165    Level 2: Describe the formation of ionic bond ...
615166    Level 3: Describe the formation of ionic bond ...
615167    Level 1: Describe the formation of ionic bond ...
615168             Ligações iônicas, covalentes e metálicas
615169                Électronégativité et liaison chimique
Name: content_title, Length: 615161, dtype: object

In [170]:
content_title.nunique()

82445

In [177]:
content_title.str.len().min()

2

* special characters
* numbers and book sections like `Chapter 103` or `4.2:`
* lots of content titles overlap

In [178]:
start = int(input("Enter a start: "))

content_title.unique()[start:start+500]

Enter a start:  82000


array(['La transformada de "desplazamiento" mediante la multiplicación de una función por una exponencial',
       'Lab 1: Introduction to the Microscope and Comparison of Sizes and Shapes of Microorganisms',
       'Lab 1: Introduction to the Microscope and Comparison of Sizes and Shapes of Microorganisms_Lab_1_selfquiz.pdf',
       '3.1: Observing microorganisms with light microscopy',
       'يوجد النسبة التي تقسم بها قطعة مستقيمة من الداخل أو الخارج إذا علم نهايتا القطعة المستقيمة التمرين',
       'સમાંતર બાજુ ચતુષ્કોણના બાજુ અને ખૂણાના ગુણધર્મ (લેવલ 1)',
       '4. Planning and using TESSA materials with student teachers',
       '4. Planifier et utiliser les matériels TESSA avec les enseignants',
       'Student Research Plan Handout_10.3.2.L2.pdf',
       'Student Research Plan Handout_11.3.2.L2.pdf',
       'Triangles et quadrilatères - Losange (cours)',
       'Solve word problems on linear equation in one variable- variable on both sides',
       'Solve word problems on linea

# Content Description

* same as before

In [183]:
content_description = train_context.content_description[~train_context.content_description.isna()]

In [184]:
content_description

1         Recurso Educativo Abierto para Ciencias Natura...
2         Recurso Educativo Abierto para Geografía e His...
6         Recurso Educativo Abierto para Geografía e His...
9         ડેવિડ સમજાવે છે કે લોલકને સરળ આવર્ત દોલક તરીકે...
11        Nesta videoaula, apresentamos uma distinção or...
                                ...                        
615162    Разлики в електроотрицателността в химичните в...
615163    Защо отклонението се променя, когато се свърже...
615164    Electronegativity differences in bonding using...
615168    Existem três tipos principais de ligações quím...
615169    Différences d'électronégativité au sein d'une ...
Name: content_description, Length: 358621, dtype: object

# Content Text

* same as previous

In [187]:
content_text = train_context.content_text[~train_context.content_text.isna()]

In [188]:
content_text

0         Orientaciones para el profesorado\nOrientacion...
1         ¿Es magia? No, son reacciones químicas\n¿Es ma...
2         La economía y yo\nLa economía y yo\nObra publi...
3         La curiosidad es saludable\nLa curiosidad es s...
4         Palabra de caballero\nPalabra de caballero\n"P...
                                ...                        
615139    Skills to Develop\n\nExplain how hormonal cues...
615140    (clacking) (metal clattering) (coin clattering...
615150    (clacking) (metal clattering) (coin clattering...
615162    Електроотрицателност е може би най-важната кон...
615164    Electronegativity is probably the most importa...
Name: content_text, Length: 243294, dtype: object

# topic_parent_title

* same as before
* perhaps for topics, only parent/children nodes with the same language should be considered (for both training and inference)
* maybe develop some method to auto select the best parent/children candidates in helping match to content
* maybe develop a method to decide when to use description, text, parent/children description or title

In [195]:
topic_parent_title = train_context.topic_parent_title[~train_context.topic_parent_title.isna()]

In [196]:
len(topic_parent_title)

615070

In [198]:
topic_parent_title.nunique()

14700

How come there are so few unique cases?

In [200]:
start = int(input("Enter a start: "))

topic_parent_title.unique()[start:start+500]

Enter a start:  14500


array(['1.1.3 Conductors and non-conductors of electricity [SEP] 1.1 Review of matter, mixtures, conduction, drugs [SEP] 1.0 Introduction to Chemistry (12 lessons) [SEP] Chemistry [SEP] Form 1 [SEP] Kolibri Library for Kenya [SEP]',
       '5: Gas Laws [SEP] Bucknell University [SEP] Campus Bookshelves [SEP] LibreTexts Engineering [SEP]',
       'Strand 8: Demonstrate understanding in asking and answering questions [SEP] Strand 1: Oral language [SEP] B3 English Language Curriculum [SEP] B1 - B3 English Language Curriculum [SEP] Ghana English Language Curriculum [SEP]',
       'Sub-Strand 5: Using Qualifying Words-Adjectives [SEP] Strand 5 [SEP] B5 English Language Curriculum [SEP] B4 - B6 English Language Curriculum [SEP] Ghana English Language Curriculum [SEP]',
       '9.7.3 Observe permanent slides of mammalian skin [SEP] 9.7 Practical activities [SEP] 9.0 Excretion and homeostasis (42 lessons) [SEP] Biology [SEP] Form 2 [SEP] Kolibri Library for Kenya [SEP]',
       'Exploramos y a

In [202]:
train_context

,topics_ids,content_ids,topic_title,topic_description,content_title,content_description,content_text,topic_parent_title,topic_parent_description,topic_child_title,topic_child_description,target
0,t_3d9ad9931021,c_efb73ad83f4b,NaN,BC: BIOL 2 - Introduction to Human Biology (Gr...,NaN,NaN,Orientaciones para el profesorado\nOrientacion...,Butte College [SEP] Campus Courses [SEP] Libre...,NaN,Text [SEP] 2: Introduction to Human Biology [S...,NaN,0
1,t_3d9ad9931021,c_77574ef20c1f,NaN,BC: BIOL 2 - Introduction to Human Biology (Gr...,NaN,Recurso Educativo Abierto para Ciencias Natura...,"¿Es magia? No, son reacciones químicas\n¿Es ma...",Butte College [SEP] Campus Courses [SEP] Libre...,NaN,Text [SEP] 2: Introduction to Human Biology [S...,NaN,0
2,t_3d9ad9931021,c_200ae87baf4d,NaN,BC: BIOL 2 - Introduction to Human Biology (Gr...,NaN,Recurso Educativo Abierto para Geografía e His...,La economía y yo\nLa economía y yo\nObra publi...,Butte College [SEP] Campus Courses [SEP] Libre...,NaN,Text [SEP] 2: Introduction to Human Biology [S...,NaN,0
3,t_3d9ad9931021,c_87e171afe50b,NaN,BC: BIOL 2 - Introduction to Human Biology (Gr...,NaN,NaN,La curiosidad es saludable\nLa curiosidad es s...,Butte College [SEP] Campus Courses [SEP] Libre...,NaN,Text [SEP] 2: Introduction to Human Biology [S...,NaN,0
4,t_3d9ad9931021,c_3c070b63a944,NaN,BC: BIOL 2 - Introduction to Human Biology (Gr...,NaN,NaN,"Palabra de caballero\nPalabra de caballero\n""P...",Butte College [SEP] Campus Courses [SEP] Libre...,NaN,Text [SEP] 2: Introduction to Human Biology [S...,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
615165,t_70da08637930,c_eb6448437b5f,8.1.5 Use dot (.) and cross (x) diagrams to il...,Use dot (.) and cross (x) diagrams to illustra...,Level 2: Describe the formation of ionic bond ...,NaN,NaN,8.1 The role of outer electrons in chemical bo...,The role of outer electrons in chemical bondin...,8.1.5 Use dot (.) and cross (x) diagrams to il...,Materials in this folder have been reviewed by...,0
615166,t_70da08637930,c_07c1da15995b,8.1.5 Use dot (.) and cross (x) diagrams to il...,Use dot (.) and cross (x) diagrams to illustra...,Level 3: Describe the formation of ionic bond ...,NaN,NaN,8.1 The role of outer electrons in chemical bo...,The role of outer electrons in chemical bondin...,8.1.5 Use dot (.) and cross (x) diagrams to il...,Materials in this folder have been reviewed by...,0
615167,t_70da08637930,c_17ff16d31106,8.1.5 Use dot (.) and cross (x) diagrams to il...,Use dot (.) and cross (x) diagrams to illustra...,Level 1: Describe the formation of ionic bond ...,NaN,NaN,8.1 The role of outer electrons in chemical bo...,The role of outer electrons in chemical bondin...,8.1.5 Use dot (.) and cross (x) diagrams to il...,Materials in this folder have been reviewed by...,0
615168,t_70da08637930,c_7cb9a57f2219,8.1.5 Use dot (.) and cross (x) diagrams to il...,Use dot (.) and cross (x) diagrams to illustra...,"Ligações iônicas, covalentes e metálicas",Existem três tipos principais de ligações quím...,NaN,8.1 The role of outer electrons in chemical bo...,The role of outer electrons in chemical bondin...,8.1.5 Use dot (.) and cross (x) diagrams to il...,Materials in this folder have been reviewed by...,0


# Implementing Text Cleaning Pipeline

Below are all my edits.

* `:`/`?`/`!`/`.`/`'`/`-`/`(`/`)`/`/`/`,`/`<`/`>`/`=`/`^`/`\n`
* standalone non-year numbers
* '1870-1939'
* `Du +Du =D`/`DU+U/U+DU/DU+U`
* 'Lección 31', 'Lección 34', 'Lección 17', 'Lección 32',
       'Convection', 'Lección 12', 'Lección 20', 'Lección 33',
       'Lección 16'
* case vs uncased
* remove useless words (small words)
* remove white space
* handle duplicates
    * remove duplicates within same tree
    * remove 
* for context, filter out if diff language

* Need to develop some way to check whether a title is important or not (same goes for all text columns).
* maybe account for length of text (for parent/children especially)

* what is this source id thing in the topic desc?
* how do we parse book titles?
* manage length
* special symbols again
* '5ad59f7a6b9064043e263f03', '5b9e5bd46571f90499ea8d69', '5a4c844e7dd197090857fce9'
*  'નમસ્કાર, \nધો:- ૬ થી ૮ ગુજરાતીની ક્વિઝ અહિયા હુ રજૂ કરુ છું. આશા રાખુ કે આ ક્વિઝ તમને ઉપયોગી થશે. બીજી અન્ય ઉપયોગી માહિતિ મારા બ્લોગ અને યુ-ટ્યુબ ચેનલ પર જોવા  નીચેનીની લીંક પર જાઓ.                                          \n My Youtube Channel\nhttps://www.youtube.com/channel/UCQue-6mFDpz31MYWvrBDTUQ?sub_confirmation=1\nMy Blog\nhttp://mehuloza.blogspot.com/'

* special characters
* numbers and book sections like `Chapter 103` or `4.2:`
* lots of content titles overlap

* perhaps for topics, only parent/children nodes with the same language should be considered (for both training and inference)
* maybe develop some method to auto select the best parent/children candidates in helping match to content
* maybe develop a method to decide when to use description, text, parent/children description or title

In [304]:
train_context.columns

Index(['topics_ids', 'content_ids', 'topic_title', 'topic_description',
       'content_title', 'content_description', 'content_text',
       'topic_parent_title', 'topic_parent_description', 'topic_child_title',
       'topic_child_description', 'target'],
      dtype='object')

In [263]:
def clean(train_context):

    train_context = deepcopy(train_context)
    
    # Replace uninformative/duplicate titles/descriptions with nothing.
    
    # Remove stopwords.
        
    # Handle "source_id=" in topic descriptions. 
    
    # Handle literal gibberish: '5ad59f7a6b9064043e263f03' and weird links (YouTube link).
    
    # Handle Topic, Section, Chapter text.
    
    # Remove numbers.
    def remove_numbers(x):
        if x is np.nan:
            return x
        return re.sub(" \d+", "", x)
    
    # Remove special characters.
    def remove_chars(x, include_brackets=False):
        if x is np.nan:
            return x
        if include_brackets:
            return x.translate(str.maketrans('', '', string.punctuation.replace("[", "").replace("]", "")))
        return x.translate(str.maketrans('', '', string.punctuation))
            
    curr_text_cols = [
         "topic_title", 
         "topic_description", 
         "content_title", 
         "content_description", 
         "content_text"
    ]
    
    context_text_cols = [
     "topic_parent_title", 
     "topic_parent_description", 
     "topic_child_title", 
     "topic_child_description"
    ]
    
    print("Removing special characters * numbers...")
    for col in tqdm(curr_text_cols, position=0, leave=True, total=len(curr_text_cols)):
        train_context[col] = train_context[col].apply(remove_chars)
        train_context[col] = train_context[col].apply(remove_numbers)
        
    for col in tqdm(context_text_cols, position=0, leave=True, total=len(context_text_cols)):
        train_context[col] = train_context[col].apply(remove_chars, include_brackets=True)
        train_context[col] = train_context[col].apply(remove_numbers)

    

    print("Finished")
    
    # Remove leading or trailing whitespace.
    
    
    return train_context

In [264]:
tmp = clean(train_context)

Removing special characters...


  0%|          | 0/9 [00:00<?, ?it/s]

Finished


In [276]:
train_context.columns

Index(['topics_ids', 'content_ids', 'topic_title', 'topic_description',
       'content_title', 'content_description', 'content_text',
       'topic_parent_title', 'topic_parent_description', 'topic_child_title',
       'topic_child_description', 'target'],
      dtype='object')

In [287]:
print(stopwords.fileids())

['arabic', 'azerbaijani', 'basque', 'bengali', 'catalan', 'chinese', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hebrew', 'hinglish', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


In [285]:
# nltk.download('stopwords')
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each